# Neural Networks

Neural networks can be constructed using the `torch.nn` package.

Now that, you had a glimpse of `autograd`, `nn` depens on `autograd` to define models and differentiate them. An `nn.Module` contains layers, and method `forward(input)` that returns the `output`.

![](https://pytorch.org/tutorials/_images/mnist.png)

It is a simple feed-forward network. It takes the input, feeds it through several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:

* Define the neural network that has some learnable parameters(or weights)
* Iterate over a dataset inputs
* Processs input through the network
* Compute the loss (how far is the output from being correct)
* Propagate gradients back into the network's parameters
* Update the weights of the network, typically using  a simple update rule: `weight = weight - learning_rate * gradient`

## Define the network

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [20]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # an affine operation y = Wx + b
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimesion
        num_features = 1
        for s in size:
            num_features *= s
            
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


You just have to define the `forward` function, and the `backward` function (where gradients are computed) is automatically defined for you using `autograd`. You can use any of the Tensor operations in the `forward` function.

In [16]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [21]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0078, -0.0153, -0.0562, -0.0382, -0.0669,  0.0066, -0.0144,  0.0409,
          0.0010, -0.0019]], grad_fn=<AddmmBackward>)


**Zero the gradient buffers of all parameters and backprops with random gradients:**

In [23]:
net.zero_grad()
out.backward(torch.randn(1, 10))

> **`torch.nn` only supports min-batches. The entire `torch.nn` package only supports inputs that are a mini-batch of samples, and not a single sample.**

> **For exmaple, `nn.Conv2d` will takes in a 4D Tensor of `nSamples x nChannels x Height x Weight`**

> **If you have a single sample, just use the `input.unsqueeze(0)` to add a fake batch dimension.**